# Umbrella sampling

*This is the OpenMM version of Justin A. Lemkul's tutorial*

Please refer to here for the original: http://www.mdtutorials.com/gmx/umbrella/index.html  
And here for the publication: https://doi.org/10.1021/jp9110794 


## Umbrella sampling simulations

Most umbrella sampling simulations have three main steps:

1. Preparing the windows, this is usually done using steered MD.
2. Running the windows, these can be run in parallel.
3. Analysing the results, typically this involves computing a PMF with WHAM.

Often you will find in step 3 that the simulations do not produce a nicely converged PMF. You will need to go back to steps 1 and 2 and change some settings, this trial and improvement feedback loop is a normal part of the process. For this tutorial we will use settings already known to work.

## System

This tutorial will use umbrella sampling to compute the free energy of dissociation of a single peptide from the growing end of an Aβ42 protofibril. Please refer to the original [tutorial](http://www.mdtutorials.com/gmx/umbrella/01_pdb2gmx.html) and [publication](https://doi.org/10.1021/jp9110794
)


## Step 1 - setting up the windows

The script below does the following steps:
-   loads in a PDB file
-   Adds missing hyrogens
-   Solvates the protein in a water box of appropriate size
-   Adds positions restraints to chain B
-   Defines a collective variable between the center of mass of chain A and chain B
-   Adds a harmonic restraint to the CV
-   Runs a simulation where the location of the harmonic restraint is moved with contant velocity from 0.5nm to 5.0nm over 500ps (This is called constant velocity steered MD).
-   Saves a configuration for each of the 23 equally spaced windows we have defined.






In [ ]:
# first get the input file
!wget http://www.mdtutorials.com/gmx/umbrella/Files/2BEG_model1_capped.pdb

In [ ]:
"""
Script to perform constant velocity steered MD to create umbrella sampling
windows. 

This is the OpenMM version of Justin A. Lemkul's tutorial: 
http://www.mdtutorials.com/gmx/umbrella/index.html
https://doi.org/10.1021/jp9110794

"""


import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout
import numpy as np

# load in the PDB file from here:
# http://www.mdtutorials.com/gmx/umbrella/Files/2BEG_model1_capped.pdb

pdb = app.PDBFile("2BEG_model1_capped.pdb")

forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, boxSize=[7,7,12]*unit.nanometer)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

# setup the reporters
simulation.reporters.append(app.PDBReporter('smd_traj.pdb', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,time=True,
        potentialEnergy=True, temperature=True, speed=True))


# define chain A and chain B
chains = list(modeller.topology.chains())
chainA = list(atom for atom in chains[0].atoms())
chainB = list(atom for atom in chains[1].atoms())
chainA_indices = list(atom.index for atom in chainA)
chainB_indices = list(atom.index for atom in chainB)


# apply position restraints to chain B 

# define the restraint force
restraint = mm.CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
system.addForce(restraint)
restraint.addGlobalParameter('k', 100.0*unit.kilojoules_per_mole/unit.nanometer)
restraint.addPerParticleParameter('x0')
restraint.addPerParticleParameter('y0')
restraint.addPerParticleParameter('z0')

# restrain heavy atoms in chainB
for atom in chainB:
    if atom.element.symbol != "H":
        restraint.addParticle(atom.index, modeller.positions[atom.index])

# Minimize
simulation.minimizeEnergy(maxIterations=100)

# NVT equilibrate
simulation.step(1000)


# now setup SMD

# starting location of the CV
r0=0.5 # ns

# pulling force force constant
fc_pull=1000.0 # kJ/mol/nm^2

# pulling velocity (consant velocity SMD)
v_pulling=0.01 # nm / ps

# the simulation timestep
dt=0.002 # ps

# steps between incrementing the cv location
N=10

# total number of steps to run for - 500ps
M=250000

# define the collective variable as the distance between the centers
# of mass of chainA and chainB in just the z direction
cv = mm.CustomCentroidBondForce(2, 'pointdistance(0,0,z1,0,0,z2)')
cv.addGroup(chainA_indices)
cv.addGroup(chainB_indices)
cv.addBond([0, 1], [])

# define the pulling force
# we will update the value of r0 as the simulation progress 
# this is constant velocity steered MD
pullingForce = mm.CustomCVForce('0.5 * fc_pull * (cv-r0)^2')
pullingForce.addGlobalParameter('fc_pull', fc_pull)
pullingForce.addGlobalParameter('r0', r0)
pullingForce.addCollectiveVariable("cv",cv)
system.addForce(pullingForce)

simulation.context.reinitialize(preserveState=True)

# pulling loop, update CV target location every 10 steps
# save specific configurations when the current_cv_value first reaches the specified windows
windows = np.linspace(0.5,5.0,23)
window_coords = []
window_index = 0

for i in range(M//N):
    simulation.step(N)

    # get the value of the cv
    current_cv_value = pullingForce.getCollectiveVariableValues(simulation.context)
    
    if (i*N)%1000==0:
        print(r0, current_cv_value)

    # check if we should save this config as a window starting structure
    if (window_index < len(windows) and current_cv_value >= windows[window_index]):
        positions = simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions()
        window_coords.append(positions)
        window_index+=1

    # increment r0 based on the pulling velocity
    r0 += v_pulling* (dt*N)
    simulation.context.setParameter('r0',r0)

# save the window structures
i=0
for coords in window_coords:
    outfile = open("window_"+str(i)+".pdb","w")
    app.PDBFile.writeFile(simulation.topology,coords, outfile)
    outfile.close()
    i+=1


Once the script has completed running there will be 23 new pdb files called "window_n.pdb" where n in an integer from 0 to 22. You will also have the SMD trajectory that should look something like this:


We have now have the initial configurations for the umbrella sampling windows.

## Step 2 - running the windows

The script to run the windows is very similar to the script in step 1. The key differences are that we load in an initial structure that corresponds to each specific window and the harmonic restraint on the CV does not move. The script below will run a single window for 10ns.

In [ ]:

"""

Script to run a single umbrella sampling window

This is the OpenMM version of Justin A. Lemkul's tutorial: 
http://www.mdtutorials.com/gmx/umbrella/index.html
https://doi.org/10.1021/jp9110794


"""

import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout
import numpy as np
import sys


windows = np.linspace(0.5,5.0,23)

window_number = int(sys.argv[1])

input_pdb_file = "window_"+str(window_number)+".pdb"
outtraj = "window_"+str(window_number)+"_traj.pdb"

# use the same setup as before
pdb = app.PDBFile(input_pdb_file)
# chose the forcefield
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
#simulation.reporters.append(app.PDBReporter(outtraj, 10000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,time=True,
        potentialEnergy=True, temperature=True, speed=True))
# define chain A and chain B
chains = list(pdb.topology.chains())
chainA = list(atom for atom in chains[0].atoms())
chainB = list(atom for atom in chains[1].atoms())
chainA_indices = list(atom.index for atom in chainA)
chainB_indices = list(atom.index for atom in chainB)
# apply position restraints to chain B 
# define the restraint force
restraint = mm.CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
system.addForce(restraint)
restraint.addGlobalParameter('k', 100.0*unit.kilojoules_per_mole/unit.nanometer)
restraint.addPerParticleParameter('x0')
restraint.addPerParticleParameter('y0')
restraint.addPerParticleParameter('z0')
# restrain heavy atoms in chainB
for atom in chainB:
    if atom.element.symbol != "H":
        restraint.addParticle(atom.index, pdb.positions[atom.index])

# now setup Umbrella window
r0 = windows[window_number]
fc_pull=1000.0 # kJ/mol/nm^2

# define the collective variable as the distance between the centers
# of mass of chainA and chainB
cv = mm.CustomCentroidBondForce(2, 'pointdistance(0,0,z1,0,0,z2)')
cv.addGroup(chainA_indices)
cv.addGroup(chainB_indices)
cv.addBond([0, 1], [])
cv.setUsesPeriodicBoundaryConditions(True)

# define the harmonic restraint
pullingForce = mm.CustomCVForce('0.5 * fc_pull * (cv-r0)^2')
pullingForce.addGlobalParameter('fc_pull', fc_pull)
pullingForce.addGlobalParameter('r0', r0)
pullingForce.addCollectiveVariable("cv",cv)
system.addForce(pullingForce)

simulation.context.reinitialize(preserveState=True)

# now run MD with the biasing force for 10ns
# record the CV timeseries
cv_values=[]
for i in range(5000):
    simulation.step(1000)

    # get the value of the cv
    current_cv_value = pullingForce.getCollectiveVariableValues(simulation.context)
    cv_values.append([i, current_cv_value[0]])

    # save the cv timeseries to file
    if i%10==0:
        np.savetxt("cv_values_window_"+str(window_number)+".txt",np.array(cv_values))

np.savetxt("cv_values_window_"+str(window_number)+".txt",np.array(cv_values))


You will need to run this script for each window. These can be run in parallel, e.g.

```bash
#!/bin/bash
# run_all_windows.sh

for i in {0..22}
do
    python run_window.py ${i} &
done
wait
```

In [ ]:
!bash run_all_windows.sh

Once all the window simulations have completed you will have the CV timeseries files: "cv_values_window_n.txt"

## Step 3 - analysis - compute the PMF

The first thing to check is that the histrograms of the CV timeseries from the windows have good overlap. Here is an example:
![histogram](histograms.png)


You can plot yours with the script below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the histograms
windows = np.linspace(0.5,5.0,23)
for i in range(22):
    data=np.loadtxt("cv_values_window_"+str(i)+".txt")
    plt.hist(data[:,1],bins=20)
plt.show()


To compute the PMF we can use WHAM. An easy to use and widely compatible implementation is the WHAM program by Alan Grossfield, it can be downloaded here: http://membrane.urmc.rochester.edu/?page_id=126


In [ ]:
!wget http://membrane.urmc.rochester.edu/sites/default/files/wham/wham-release-2.0.11.tgz
!tar xvf wham-release-2.0.11.tgz
!cd wham/wham && make

To use wham we need a metadata file that lists the names of each CV timeseries file, the location of the harmonic restraint and the value of the spring constant. We have provided that in "metafile.txt".

The `wham` program is run using command line arguments, read the documentation to find out more: http://membrane.urmc.rochester.edu/sites/default/files/wham/doc.pdf

The command below will compute the PMF from our data in the range 0.5nm to 5.0nm, with 50 bins, a tolerance of 1e-6 for our simulated temperature of 300K.

In [ ]:
!./wham/wham/wham 0.5 5.0 50 1e-6 300 0 metafile.txt pmf.txt

We can then plot the computed PMF. It should look something like this:
![pmf.png](pmf.png)

In [ ]:
# plot the pmf
pmf=np.loadtxt("pmf.txt")
plt.plot(pmf[:,0], pmf[:,1])
plt.xlabel("z (nm)")
plt.ylabel("PMF (kJ/mol)")
plt.show()